In [ ]:
from pathlib import Path
import time
import pandas as pd
import geopandas as gpd
import numpy as np
import pickle
import networkx as nx
from stochopy.optimize import minimize
from tqdm import tqdm
import similaritymeasures
import random
import matplotlib.pyplot as plt
from shapely.ops import Point, MultiLineString, LineString
from importlib import reload
import datetime
from scipy.spatial.distance import directed_hausdorff
np.set_printoptions(suppress=True)

from bikewaysim.paths import config, stadia_toner, maptiler_streets
from bikewaysim.impedance_calibration import stochastic_optimization, speedfactor
from bikewaysim.network import modeling_turns
from bikewaysim.routing import rustworkx_routing_funcs
from bikewaysim.impedance_calibration import optimization_viz, utils, post_calibration

from step_1_calibration_experiments import all_calibrations, full_model

## Post Validation
- Retrieve loss values for testing and training set for each fold
- Create confidence intervals of the betas from the bootstrapped results

## K-fold Results

In [ ]:
loss_vals = post_calibration.aggregated_loss_dataframe()
training_loss_vals = loss_vals[loss_vals['subset'].str.contains('fold_')]

testing_loss_vals = post_calibration.testing_aggregated_loss_dataframe()

# print the results of each
print("Training Overlap Values:",str(training_loss_vals.sort_values('subset')['jaccard_buffer_mean'].tolist()))
print("Testing Overlap Values:",str(testing_loss_vals.sort_values('subset')['jaccard_buffer_mean'].tolist()))

# report the averages
print("Training Overlap Mean:",training_loss_vals['jaccard_buffer_mean'].mean().round(2))
print("Testing Overlap Mean:",testing_loss_vals['jaccard_buffer_mean'].mean().round(2))

## Bootstrapping Results

In [ ]:
# import all of the betas
beta_vals = post_calibration.betas_dataframe()

# only want the bootstrap results
beta_vals = beta_vals[beta_vals['subset'].str.contains('bootsample')].dropna(axis=1,how='all')

In [ ]:
# get the beta statistics
betas = ['2lpd','3+lpd','(30,inf) mph','[4,6) grade','[6,inf) grade','bike lane','cycletrack','multi use path','unsig_crossing','left_turn','right_turn']
beta_vals[betas].describe()

In [ ]:
# plot histograms
beta_vals.hist('2lpd')